In [1]:
import json
import csv
import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [2]:
df1=pd.read_json('iiitb.json')
df2=pd.read_json('iiitb2.json')

In [3]:
tweets_df = pd.concat([df1,df2], axis=0)
tweets_df = tweets_df.loc[:, ['text' , 'username' ,'tweet_id', 'timestamp' ,'hashtags' , 'screen_name','retweets','likes']]
tweets_df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes
0,"@IIITB_official Director, Prof Sadagopan welco...","Consulate General of France, Bangalore",1210499859700011008,2019-12-27 09:57:11,"[collaboration, academic, exchanges, startup, ...",BangaloreFrance,0,0
1,May the peace and blessings of Christmas be yo...,IIIT Bangalore,1209693257443110912,2019-12-25 04:32:02,"[Christmas, MerryChristmas, IIITB, IIITB10100,...",IIITB_official,1,6
2,Wishing everyone Merry Christmas & Happy holid...,Agroblock,1209470538738913280,2019-12-24 13:47:01,"[christmas, agritech, happyholidays, iiitb, cc...",agroblock_io,0,1
3,Our Ph.D. student Mr. Shyam S.M. has been sele...,IIIT Bangalore,1209094761589592064,2019-12-23 12:53:49,"[IIITB, IIITB10100pic]",IIITB_official,5,12
4,AgroBlock wishes Kisan Diwas to all the farmer...,Agroblock,1209072927921958912,2019-12-23 11:27:04,"[kisandiwas, farmersday, agritech, blockchaint...",agroblock_io,0,4
5,Superb #Kohli #kohlidecadehttps://twitter.com/...,Harsh Dubey,1209069978479030272,2019-12-23 11:15:20,"[Kohli, kohlidecadehttps]",harsh_iiitb,0,0
6,"Love your work, Veena! You’re the best ... @An...",Gerard Shanahan,1208283808434274304,2019-12-21 07:11:23,[],gas3040,0,2
7,"Our last #GLE for 2019, in my home City Bangal...",Veena,1208184116614578176,2019-12-21 00:35:15,"[GLE, GLELeaders]",veena_haran,0,8
8,But IIITB has 2000 students?,indian,1207827046862331904,2019-12-20 00:56:22,[],mkinthuk,0,0
9,"@IIITB_official Director, Prof Sadagopan welco...","Consulate General of France, Bangalore",1210499859700011008,2019-12-27 09:57:11,"[collaboration, academic, exchanges, startup, ...",BangaloreFrance,0,0


In [4]:
tweets_df.reset_index(drop=True, inplace=True)
tweets_df = tweets_df[~tweets_df.index.duplicated()]
uni_tweets = tweets_df.loc[tweets_df['screen_name'].isin(['IIITB_official','SpandanIIITB','Infin8_IIITB'])]
uni_tweets['uni_handle'] = 1
uni_tweets

/Users/Manam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle
1,May the peace and blessings of Christmas be yo...,IIIT Bangalore,1209693257443110912,2019-12-25 04:32:02,"[Christmas, MerryChristmas, IIITB, IIITB10100,...",IIITB_official,1,6,1
3,Our Ph.D. student Mr. Shyam S.M. has been sele...,IIIT Bangalore,1209094761589592064,2019-12-23 12:53:49,"[IIITB, IIITB10100pic]",IIITB_official,5,12,1
10,May the peace and blessings of Christmas be yo...,IIIT Bangalore,1209693257443110912,2019-12-25 04:32:02,"[Christmas, MerryChristmas, IIITB, IIITB10100,...",IIITB_official,1,6,1
12,Our Ph.D. student Mr. Shyam S.M. has been sele...,IIIT Bangalore,1209094761589592064,2019-12-23 12:53:49,"[IIITB, IIITB10100pic]",IIITB_official,5,12,1
25,"Sridhar Varadharajan, Preeti Mudliar and Balaj...",IIIT Bangalore,1200344006435074048,2019-11-29 09:21:27,[],IIITB_official,5,23,1
34,"Sridhar Varadharajan, Preeti Mudliar and Balaj...",IIIT Bangalore,1200344006435074048,2019-11-29 09:21:27,[],IIITB_official,5,23,1
39,"AI is hot, should be handled with care: Dr. Ba...",IIIT Bangalore,1164505812934385664,2019-08-22 11:53:15,"[IIITB10100, IIITBangalore, IIITB, BangaloreOn...",IIITB_official,2,10,1
40,"Speaker: Prof. Ashish Choudhury, IIIT Bangalor...",IIIT Bangalore,1164463621167960064,2019-08-22 09:05:36,"[IIITB10100, SamvaadTalkpic]",IIITB_official,0,1,1
41,The first sports event of the academic year!!\...,IIIT Bangalore,1162621639390986240,2019-08-17 07:06:13,"[GameOn, BPL, IIITB10100pic]",IIITB_official,0,2,1
42,May the glory of Independence Day be with us f...,IIIT Bangalore,1161869416885149696,2019-08-15 05:17:10,"[IndependenceDay2019, HappyIndependenceDay, II...",IIITB_official,0,1,1


In [5]:
tweets_df = tweets_df[~tweets_df.index.duplicated()]
tweets_df['uni_handle'] = uni_tweets['uni_handle']
df = tweets_df.combine_first(tweets_df)
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle
0,"@IIITB_official Director, Prof Sadagopan welco...","Consulate General of France, Bangalore",1210499859700011008,2019-12-27 09:57:11,"[collaboration, academic, exchanges, startup, ...",BangaloreFrance,0,0,NaN
1,May the peace and blessings of Christmas be yo...,IIIT Bangalore,1209693257443110912,2019-12-25 04:32:02,"[Christmas, MerryChristmas, IIITB, IIITB10100,...",IIITB_official,1,6,1.0
2,Wishing everyone Merry Christmas & Happy holid...,Agroblock,1209470538738913280,2019-12-24 13:47:01,"[christmas, agritech, happyholidays, iiitb, cc...",agroblock_io,0,1,NaN
3,Our Ph.D. student Mr. Shyam S.M. has been sele...,IIIT Bangalore,1209094761589592064,2019-12-23 12:53:49,"[IIITB, IIITB10100pic]",IIITB_official,5,12,1.0
4,AgroBlock wishes Kisan Diwas to all the farmer...,Agroblock,1209072927921958912,2019-12-23 11:27:04,"[kisandiwas, farmersday, agritech, blockchaint...",agroblock_io,0,4,NaN
5,Superb #Kohli #kohlidecadehttps://twitter.com/...,Harsh Dubey,1209069978479030272,2019-12-23 11:15:20,"[Kohli, kohlidecadehttps]",harsh_iiitb,0,0,NaN
6,"Love your work, Veena! You’re the best ... @An...",Gerard Shanahan,1208283808434274304,2019-12-21 07:11:23,[],gas3040,0,2,NaN
7,"Our last #GLE for 2019, in my home City Bangal...",Veena,1208184116614578176,2019-12-21 00:35:15,"[GLE, GLELeaders]",veena_haran,0,8,NaN
8,But IIITB has 2000 students?,indian,1207827046862331904,2019-12-20 00:56:22,[],mkinthuk,0,0,NaN
9,"@IIITB_official Director, Prof Sadagopan welco...","Consulate General of France, Bangalore",1210499859700011008,2019-12-27 09:57:11,"[collaboration, academic, exchanges, startup, ...",BangaloreFrance,0,0,NaN


In [6]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
df['compound'] = [analyzer.polarity_scores(v)['compound'] for v in df['text']]

In [7]:
def sentiment(compound):
    res = ""
    if(compound >= 0.05):
        res="Positive"
    elif(compound <= - 0.05):
        res="Negative"
    else:
        res="Neutral"
    return res

df['Sentiment'] = df['compound'].apply(lambda x: sentiment(x))
df['score'] = pd.cut(df['compound'], bins=5, labels=[1, 2, 3, 4, 5])

In [8]:
df.to_csv('../../Datasets/Twitter/Datasets with likes/final_iiitb_withlikes.csv')